In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import Counter

import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.corpus import stopwords

from random import randint
import string
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
from jupyterthemes import jtplot
from nltk.stem.porter import PorterStemmer

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Javascript, display
from ipywidgets import widgets

In [3]:

#read in one dataset
# only take in the columns i need
df=pd.read_csv('/Users/denisdunn/flatiron/Recipes_Project/epicurious')

df=df[['directions','ingredients','title']]

df=df.reset_index(drop=True)

df=df.dropna()


In [4]:
# get stop words


#stopwords_list = stopwords.words('english')

#add punctuation to stopwords list
#stopwords_list += list(string.punctuation)

#add words that aren't in the NLTK stopwords list
#stopwords_list += ['unsalted','grated','minced','ground','thinly','ounces','finely','black','red','white','plus','whole','coursely','cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
#stopwords_list+=['``',"'s","''",'drained','baking','coarsely','stick','green','packed','all-purpose','room','temperature','1/2-inch','''''','pounds','dry','divided','halved','trimmed','slices','1/2-inch-thick','1/4-inch-thick','s','12','1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','1/8','small','large','medium']
#created a function to process the columns 
def process_article(article):
    stopwords_list = stopwords.words('english')

#add punctuation to stopwords list
    stopwords_list += list(string.punctuation)

#add words that aren't in the NLTK stopwords list
    stopwords_list += ['unsalted','grated','minced','ground','thinly','ounces','finely','black','red','white','plus','whole','coursely','cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
    stopwords_list+=['``',"'s","''",'drained','baking','coarsely','stick','green','packed','all-purpose','room','temperature','1/2-inch','''''','pounds','dry','divided','halved','trimmed','slices','1/2-inch-thick','1/4-inch-thick','s','12','1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','1/8','small','large','medium']

    tokens = word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

In [5]:
# created columns that i can vectorize 
df['processed_titles']=df['title'].apply(lambda x: process_article(x))

# this is the best format so far
df['processed_ingredients']=df['ingredients'].apply(lambda x: process_article(x))

df['processed_directions']=df['directions'].apply(lambda x: process_article(x))
# made a column of all the words in a recipe that im using to model
df['recipe_combo']=df['processed_titles']+df['processed_ingredients']+df['processed_directions']
#built a model using word2vec
# 2 neural networks, i used CBOW to vectorize the words
# using cosine similarity to build the relationships
model_combo = Word2Vec(df['recipe_combo'], min_count=1)
    

In [6]:
# build a model to test predicting words in titles
title_model=Word2Vec(df['processed_titles'],min_count=1)

In [7]:
# also testing a model to predict ingredients
ingredient_model=Word2Vec(df['processed_ingredients'],min_count=1)

In [8]:
# vectorized each recipe using my model
# reduced dimensionality by summing the vectors of each recipe together
df['recipe_vector']=df['recipe_combo'].apply(lambda x: sum(model_combo.wv[x]))
# create new dataframe with vectors and reindexed
recipe=df[['title','recipe_vector','ingredients','processed_ingredients','directions']]
recipe=recipe.reset_index(drop=True)
#created a list of the vectors and turned them into an array
# got the right shape for the kmeans clustering
test_cluster=[]
for num in range(len(recipe['recipe_vector'])):
    test_cluster.append(recipe['recipe_vector'][num])
X=np.asarray(test_cluster)
   

In [9]:
# function to make a button run cells for the science fair guests

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Make Unique Recipe")
button.on_click(run_all)
display(button)

Button(description='Make Unique Recipe', style=ButtonStyle())

In [10]:
# alternative function to running specific cells
def run_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

click = widgets.Button(description="Create next input")
click.on_click(run_next)
display(click)

Button(description='Create next input', style=ButtonStyle())

In [11]:
# function to filter clustered groups by specific ingredients up to 2 different ingredients
def recipe_filter(cluster_book,ingred_1,ingred_2=None):
    if ingred_2==None:
        ingred_2=ingred_1
    test_run=cluster_book[cluster_book['ingredients'].str.contains(ingred_1)]
    test_run=test_run[test_run['ingredients'].str.contains(ingred_2)]
    test_run=test_run.reset_index(drop=True)
    return test_run

In [12]:
# I chose Kmeans clustering, it is unsupervised learning.
#i tested a few other cluster methods in other notebooks
# i add in the index after the cluster to see what is in each cluster

def recipe_clusterer(num):
    k_means = KMeans(n_clusters = num)
    k_means.fit(X)
    y_hat = k_means.predict(X)
    cluster_numbers=[]
    for number in range(len(y_hat)):
        cluster_numbers.append(y_hat[number])
    recipe['clustered_groups']=pd.DataFrame(cluster_numbers)
    return recipe

In [13]:
# function to make sure computer doesnt replace the ingredients we want in recipe
def ingredient_replacements(test_ingredients,ingred_1=None,ingred_2=None):
    if ingred_2==None:
        ingred_2=ingred_1
    if ingred_1==None:
        return test_ingredients
    else:
        replacement_ingredients=[]
        count=0

        for food in range(len(test_ingredients)):
            if test_ingredients[food]==ingred_1 or test_ingredients[food]==ingred_2:
                continue
            else:
                replacement_ingredients.append(test_ingredients[food])
                count+=1
        return replacement_ingredients

In [14]:
# function to replace ingredients in recipe using cosine similarity with added layer
def ingredient_generator(replacement_ingredients):
    out_word=replacement_ingredients[0]
    answer=model_combo.wv.most_similar(positive=[out_word])

    final=answer[0][0]

    similar_words=model_combo.wv.most_similar(positive=[out_word])
# built additional layer to run list of similar words against the top similar word
#found better results with this layer
    similar_list=[]
    words=[]
    for word in range(1,len(similar_words)):
        similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
        words.append(similar_words[word][0])

    layered_relationships=dict(zip(words,similar_list))

    final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))
    return out_word, final_2

In [15]:
# replace words in actual recipe
def replace(word,ingred,replacement):
    return word.replace(ingred,replacement)

In [16]:
# built a side function to play around with ingredient substitutes
def ingredient_substitute(food):
    try:
        food_list=[]
        stemmer = PorterStemmer()
        sub=ingredient_model.wv.most_similar([food])
        for ingred in range(len(sub)):
            food_list.append(sub[ingred][0])
        food_list=[stemmer.stem(plural) for plural in food_list]
        
        return food_list 
    except:
        return print('We only substitute real food')

In [17]:
# function to that displayed the final recipe
def unique_recipe(test_ingredients,ingred_1,ingred_2):
    shopping=ingredient_replacements(test_ingredients,ingred_1,ingred_2)

    finish=replace(finished_product,ingredient_generator(shopping)[0],ingredient_generator(shopping)[1])

    two_removed=ingredient_replacements(shopping,ingredient_generator(shopping)[1])

    tester=replace(finish,ingredient_generator(two_removed)[0],ingredient_generator(two_removed)[1])

    tester=tester.split('[')
    return tester

In [18]:
# built function that displays word clouds of the top 100 words in the titles of recipes
# for cluster. found that title column was best way to express the cluster
def word_clouder(num=0):
    group=recipe[recipe['clustered_groups']==num]['title']
    processed_words=group.apply(lambda x: process_article(x))
    articles_concat = []
    for article in processed_words:
        articles_concat += article
    word_list=Counter(articles_concat).most_common(100)
    word_list_2=dict(word_list)
    wordcloud = WordCloud().generate_from_frequencies(word_list_2)
    plt.figure(figsize=[20,10])
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    return plt.show()


In [198]:

recipe_clusterer(1000)

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
0,"Lentil, Apple, and Turkey Wrap","[-24.390846, -48.1365, -1.0775162, 15.221485, ...",['4 cups low-sodium vegetable or chicken stock...,"[low-sodium, vegetable, chicken, stock, dried,...","['1. Place the stock, lentils, celery, carrot,...",674
1,Boudin Blanc Terrine with Red Onion Confit,"[-50.241173, -82.65877, 28.327993, 105.734245,...","['1 1/2 cups whipping cream', '2 medium onions...","[whipping, cream, onions, bay, leaves, cloves,...",['Combine first 9 ingredients in heavy medium ...,191
2,Potato and Fennel Soup Hodge,"[-43.550987, -17.953466, 52.904842, 6.8840103,...","['1 fennel bulb (sometimes called anise), stal...","[fennel, bulb, sometimes, called, anise, stalk...",['In a large heavy saucepan cook diced fennel ...,383
3,Mahi-Mahi in Tomato Olive Sauce,"[-66.119804, 7.9717336, 0.11911702, 68.38251, ...","['2 tablespoons extra-virgin olive oil', '1 cu...","[extra-virgin, olive, oil, onion, wine, anchov...",['Heat oil in heavy large skillet over medium-...,348
4,Spinach Noodle Casserole,"[10.73018, 0.89205843, 26.813995, 34.522057, 0...","['1 12-ounce package frozen spinach soufflé, t...","[12-ounce, package, frozen, spinach, soufflé, ...",['Preheat oven to 350°F. Lightly grease 8x8x2-...,122
5,The Best Blts,"[-29.90899, -38.19793, 9.25085, 24.720531, 24....",['2 1/2 cups (lightly packed) fresh basil leav...,"[lightly, basil, leaves, mayonnaise, butter, '...","['Mix basil, mayonnaise and butter in processo...",263
6,Ham and Spring Vegetable Salad with Shallot Vi...,"[9.877289, -24.048481, 21.41646, -26.539988, -...","['1 1/2 pounds small red-skinned potatoes, eac...","[red-skinned, potatoes, wedges, baby, carrots,...",['Cook potatoes and carrots in large pot of bo...,176
7,Spicy-Sweet Kumquats,"[-16.387747, 29.568743, 7.0735416, 49.880653, ...","['6 tablespoons granulated sugar', '1 1/2 tabl...","[granulated, sugar, chili, powder, powdered, e...",['Stir together sugar and chili powder. Whisk ...,313
8,Korean Marinated Beef,"[-47.972134, 46.14586, 26.34367, -14.659713, 1...","['1/4 cup soy sauce', '1 tablespoon sugar', '2...","[soy, sauce, sugar, asian, sesame, oil, scalli...","['Stir together soy sauce, sugar, sesame oil, ...",97
9,Ham Persillade with Mustard Potato Salad and M...,"[21.311316, -62.372997, 19.809553, 23.328907, ...","['6 long parsley sprigs, divided', '1 3/4 cups...","[long, parsley, sprigs, reduced-sodium, chicke...",['Chop enough parsley leaves to measure 1 tabl...,831


# THE MACHINE LEARNING RECIPE GENERATOR

In [202]:
# widget that plays each word cloud for the science fair
play = widgets.Play(
interval=2000,
value=0,
min=0,
max=1000,
step=1,
description="Press play",
disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play,slider])
interact(word_clouder,num=slider)
interact_manual(word_clouder,num=play)

interactive(children=(IntSlider(value=0, description='num'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Play(value=0, description='Press play', interval=2000, max=1000), Button(description='Ru…

<function __main__.word_clouder(num=0)>

# FIRST INGREDIENT YOU WANT IN RECIPE

In [191]:
ingred_1=input('type something:'
              )

type something:


# SECOND INGREDIENT YOU WANT IN RECIPE

In [41]:
ingred_2=input('type something:')

type something:


In [192]:
button

<IPython.core.display.Javascript object>

Button(description='Make Unique Recipe', style=ButtonStyle())

In [193]:
# connected wordcloud to the recipes dataframe 
cluster_book=recipe[recipe['clustered_groups']==slider.value]

filtered_recipe_book=(recipe_filter(cluster_book,ingred_1,ingred_2))
# took random recipe out of dataframe with necessary columns
data=filtered_recipe_book
item=randint(0,(len(data)-1))
test_ingredients=data['processed_ingredients'][item]
title=data.iloc[item]['title']
ingredients=data.iloc[item]['ingredients']
directions=data.iloc[item]['directions']
finished_product=title+ingredients+directions

bon_apetite=unique_recipe(test_ingredients,ingred_1,ingred_2)

In [194]:
ted={'TITLE': bon_apetite[0],'DIRECTIONS': bon_apetite[2],'INGREDIENTS':bon_apetite[1]}

In [195]:
# slightly better display 
out = widgets.Output(layout={'border': '1px solid green'})
with out:
    display(ted)
    display(word_clouder(slider.value))

# UNIQUE RECIPE!!!!

In [196]:
out

Output(layout=Layout(border='1px solid green'))

# BETA TESTING 

In [197]:
ingredient_substitute('onion')

['onion',
 'bell',
 'zucchini',
 'carrot',
 'carrot',
 'shallot',
 'seed',
 'shallot',
 'summer',
 'garlic']